<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/Intro-to-GANs/blob/master/Conditional_GAN/Simple_CGAN_with_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN (CGAN)
### (with prediction of the label)

This is a simple Conditional GAN based on the same template as [this](https://colab.research.google.com/drive/1RxJBQQJf7tszqFyKte8jsFtOF-oE89Y9) one. The difference is that in this case, we don't pass the label to the discriminator as an input, but instead we ask from the discriminator to find the most suitable label for the given image.

### Imports

In [0]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Reshape, Flatten
from keras.layers import Embedding, multiply, add, concatenate
from keras.utils.np_utils import to_categorical #NEW!
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist, fashion_mnist
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

Using TensorFlow backend.


### Function to build the generator

The generator is the same as in the previous case

In [0]:
def build_generator(noise_size, img_shape, classes_num):
  
  noise = Input((noise_size,))
  label = Input((1,), dtype='int32')
  
  embedded_label = Flatten()(Embedding(classes_num, noise_size)(label))
  model_input = concatenate([noise, embedded_label])
  
  x = Dense(256)(model_input)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(512)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(1024)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(np.prod(img_shape), activation='tanh')(x)
  img = Reshape(img_shape)(x)
    
  generator = Model([noise, label], img)
  return generator

### Function to build the discriminator

In this case the discriminator takes one input (the image) and returns two outputs:
- The validity of the input image (same as previous)
- The label of the image as a softmax tensor (similar to probabilities)

In [0]:
def build_discriminator(img_shape, classes_num):
    
  img = Input(img_shape)
  f_img = Flatten()(img)
  
  x = Dense(1024)(f_img) #CHANGED!
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(512)(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(256)(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  validity = Dense(1, activation='sigmoid')(x)
  label = Dense(classes_num, activation='softmax')(x) #NEW!
  
  discriminator = Model(img, [validity, label]) #CHANGED!
  return discriminator

### Function to compile the models

In [0]:
def get_compiled_models(generator, discriminator, noise_size):
  
  optimizer = Adam(0.0002, 0.5)
  
  discriminator.compile(optimizer,
                        loss=['binary_crossentropy', 'categorical_crossentropy'], #CHANGED!
                        metrics=['accuracy'])
  discriminator.trainable = False
  
  noise = Input((noise_size,))
  label = Input((1,), dtype='int32')
  img = generator([noise, label])
  validity, d_label = discriminator(img) #CHANGED!
  combined = Model([noise, label], [validity, d_label]) #CHANGED!
  
  combined.compile(optimizer, loss=['binary_crossentropy', 'categorical_crossentropy']) #CHANGED!
  
  return generator, discriminator, combined

### Function to sample and save generated images

In [0]:
def sample_imgs(generator, noise_size, step, classes_num, plot_img=True):
  
  r, c = classes_num, 5
  imgs = []
  for i in range(c):
    noise = np.random.normal(0, 1, (r, noise_size))
    sampled_labels = np.arange(r).reshape(-1, 1)
    img = generator.predict([noise, sampled_labels])
    img = img / 2 + 0.5
    imgs.append(img)
  
  figsize = 1 * c, 1 * r
  fig, axs = plt.subplots(r, c, figsize=figsize)
  
  for i in range(r):
    for j in range(c):
      axs[i, j].imshow(imgs[j][i], cmap='gray')
      axs[i, j].axis('off')
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  fig.savefig(f'/content/images/{step}.png')
  if plot_img:
    plt.show()
  plt.close()

### Function to train the models

The training follows the same pattern as in the previous case. This time, the discriminator has one input and two outputs, while the combined model two inputs and two outputs. Pay attetion to the fact that the input label is a single integer, while the target label is a [one-hot](https://keras.io/utils/#to_categorical) vector.

In [0]:
def train(models, noise_size, img_shape, batch_size, steps, classes_num):
  
  generator, discriminator, combined = models
  #get real data
  (X_train, Y_train), (X_val, Y_val) = fashion_mnist.load_data()
  mnist_imgs = np.concatenate((X_train, X_val)) / 127.5 - 1 
  mnist_labels = np.concatenate((Y_train, Y_val))
  
  for step in range(1, steps + 1):
    # train discriminator
    inds = np.random.randint(0, mnist_imgs.shape[0], batch_size)
    real_imgs, labels = mnist_imgs[inds], mnist_labels[inds]
    real_validity = np.ones(batch_size)
    
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_imgs = generator.predict([noise, labels])
    gen_validity = np.zeros(batch_size) 

    one_hot_labels = to_categorical(labels, num_classes=classes_num) #NEW!
    r_loss = discriminator.train_on_batch(real_imgs, [real_validity, one_hot_labels]) #CHANGED!
    g_loss = discriminator.train_on_batch(gen_imgs, [gen_validity, one_hot_labels]) #CHANGED!
    disc_loss = np.add(r_loss, g_loss) / 2
    
    # train generator
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_validity = np.ones(batch_size)
    gen_loss = combined.train_on_batch([noise, labels], [gen_validity, one_hot_labels]) #CHANGED!
    
    #print progress
    if step % 100 == 0:
      print('step: %d, D_loss: %f, D_accuracy: %d%%, '
            'D_label_accuracy: %d%%, G_loss: %f' % (step, disc_loss[0],
                                                    round(disc_loss[3] * 100), #CHANGED!
                                                    round(disc_loss[4] * 100), #CHANGED!
                                                    gen_loss[0]))
    
    # save_samples
    if step % 100 == 0:
      sample_imgs(generator, noise_size, step, classes_num)

### Define hyperparameters

In [0]:
noise_size = 100
img_shape = 28, 28
batch_size = 32
steps = 5000
classes_num = 10

### Generate the models

In [0]:
generator = build_generator(noise_size, img_shape, classes_num)
discriminator = build_discriminator(img_shape, classes_num)
compiled_models = get_compiled_models(generator, discriminator, noise_size)

### Train the models

In [0]:
%rm -r /content/images
%mkdir /content/images
train(compiled_models, noise_size, img_shape, batch_size, steps, classes_num)

### Display samples

In [0]:
Image('/content/images/%d.png' % 5000)